In [4]:
!pip install faker

In [5]:
from faker import Faker

In [6]:
import sqlite3
from faker import Faker
import random

# Create Faker instance
fake = Faker()

# Connect to SQLite
conn = sqlite3.connect("placement_eligibility.db")
cursor = conn.cursor()

# Create tables
cursor.execute('''CREATE TABLE IF NOT EXISTS students (
    student_id INTEGER PRIMARY KEY,
    name TEXT, age INTEGER, gender TEXT, email TEXT, phone TEXT,
    enrollment_year INTEGER, course_batch TEXT, city TEXT, graduation_year INTEGER
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS programming (
    programming_id INTEGER PRIMARY KEY,
    student_id INTEGER, language TEXT, problems_solved INTEGER,
    assessments_completed INTEGER, mini_projects INTEGER,
    certifications_earned INTEGER, latest_project_score INTEGER,
    FOREIGN KEY(student_id) REFERENCES students(student_id)
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS soft_skills (
    soft_skill_id INTEGER PRIMARY KEY,
    student_id INTEGER, communication INTEGER, teamwork INTEGER,
    presentation INTEGER, leadership INTEGER,
    critical_thinking INTEGER, interpersonal_skills INTEGER,
    FOREIGN KEY(student_id) REFERENCES students(student_id)
)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS placements (
    placement_id INTEGER PRIMARY KEY,
    student_id INTEGER, mock_interview_score INTEGER,
    internships_completed INTEGER, placement_status TEXT,
    company_name TEXT, placement_package INTEGER,
    interview_rounds_cleared INTEGER, placement_date TEXT,
    FOREIGN KEY(student_id) REFERENCES students(student_id)
)''')

# Insert data
for _ in range(100):
    name = fake.name()
    age = random.randint(18, 25)
    gender = random.choice(["Male", "Female", "Other"])
    email = fake.email()
    phone = fake.phone_number()
    enrollment_year = random.randint(2019, 2022)
    course_batch = "Batch " + str(random.randint(1, 5))
    city = fake.city()
    graduation_year = enrollment_year + 4

    cursor.execute("INSERT INTO students (name, age, gender, email, phone, enrollment_year, course_batch, city, graduation_year) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
                   (name, age, gender, email, phone, enrollment_year, course_batch, city, graduation_year))
    student_id = cursor.lastrowid

    cursor.execute("INSERT INTO programming (student_id, language, problems_solved, assessments_completed, mini_projects, certifications_earned, latest_project_score) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   (student_id, random.choice(["Python", "SQL", "Java"]), random.randint(10, 100), random.randint(1, 5), random.randint(0, 3), random.randint(0, 3), random.randint(60, 100)))

    cursor.execute("INSERT INTO soft_skills (student_id, communication, teamwork, presentation, leadership, critical_thinking, interpersonal_skills) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   (student_id, random.randint(50, 100), random.randint(50, 100), random.randint(50, 100), random.randint(50, 100), random.randint(50, 100), random.randint(50, 100)))

    cursor.execute("INSERT INTO placements (student_id, mock_interview_score, internships_completed, placement_status, company_name, placement_package, interview_rounds_cleared, placement_date) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                   (student_id, random.randint(40, 100), random.randint(0, 2), random.choice(["Ready", "Not Ready", "Placed"]), fake.company(), random.randint(30000, 150000), random.randint(1, 5), fake.date()))

conn.commit()
print("✅ Done! Database 'placement_eligibility.db' created with fake data.")


✅ Done! Database 'placement_eligibility.db' created with fake data.


In [7]:
import pandas as pd

query = """
SELECT s.name, s.email, p.problems_solved, ss.communication, pl.mock_interview_score
FROM students s
JOIN programming p ON s.student_id = p.student_id
JOIN soft_skills ss ON s.student_id = ss.student_id
JOIN placements pl ON s.student_id = pl.student_id
WHERE p.problems_solved > 50 AND ss.communication > 70 AND pl.mock_interview_score > 60
LIMIT 5;
"""

df = pd.read_sql_query(query, conn)
df


,name,email,problems_solved,communication,mock_interview_score
0,Kimberly Lopez,nancy89@example.org,92,85,80
1,Chad David,michele06@example.com,61,95,79
2,Benjamin Orozco,melissahughes@example.org,61,94,73
3,Debbie Martin,acostacraig@example.net,65,97,86
4,Julie Henry,ijordan@example.com,61,78,78
